In [1]:
from pathlib import Path
import itertools
from matplotlib import pyplot as plt

stim_data_dir = Path("..", "data", "Nimstim faces")
grayscale = True

open_files = [x for x in stim_data_dir.glob("*_O.bmp")]
closed_files = [x for x in stim_data_dir.glob("*_C.bmp")]

#     img = open_files[0]
data_file = list(itertools.islice(open_files, 1))[0]


In [2]:
from protosc.preprocessing import GreyScale, ViolaJones, CutCircle
from protosc.pipe_complex import PipeComplex
from protosc.feature_extraction import FourierFeatures
from protosc.io import ReadImage

In [3]:
pipe1 = ReadImage()*GreyScale()*ViolaJones()*CutCircle()*FourierFeatures()
pipe2 = ReadImage()*GreyScale()*ViolaJones(10)*FourierFeatures()
pipe_complex = PipeComplex(pipe1, pipe2)

In [4]:
pipe_complex.execute(data_file)

(200, 200, 1)


{'ReadImage+GreyScale+ViolaJones_20+CutCircle+FourierFeatures_a8s7': array([[ 4.91190792e+01-1.52831981e+03j],
        [-5.36231511e+03+3.09737553e+02j],
        [-2.87874849e+05+8.52651283e-13j],
        [-5.36231511e+03-3.09737553e+02j],
        [-2.11002459e+04+1.52831981e+03j],
        [-1.13639961e+03-2.88565938e+02j],
        [-2.97408450e+05-7.95807864e-13j],
        [-1.13639961e+03+2.88565938e+02j],
        [-1.68921157e+03+2.67709693e+02j],
        [ 6.84538296e+03-8.61564920e+02j],
        [-3.04860533e+05-1.13686838e-12j],
        [ 6.84538296e+03+8.61564920e+02j],
        [-1.68921157e+03-2.67709693e+02j],
        [ 5.76851739e+03-8.18264599e+02j],
        [-3.00977842e+05-7.24753590e-13j],
        [ 5.76851739e+03+8.18264599e+02j],
        [ 4.57737428e+02+3.11816081e+02j],
        [ 6.81953726e+03+1.30123150e+03j],
        [-3.07437053e+05+1.47792889e-12j],
        [ 6.81953726e+03-1.30123150e+03j],
        [ 4.57737428e+02-3.11816081e+02j],
        [-2.80089166e+03-9.86

In [5]:
print(pipe)

NameError: name 'pipe' is not defined

In [ ]:
images = pipe.execute(open_files, max_depth=1)

In [ ]:
images

In [ ]:
x= {None: "x"}

In [ ]:
x = [1,2,3]
x.pop()

In [ ]:
x